In [1]:
import sys
import liwc
import glob
import os
import pandas as pd
import yaml

from tqdm import tqdm

sys.path.append("../../../utils")
from absolute_path_builder import AbsolutePathBuilder

In [2]:
DATASET = "twitter"

data_path = AbsolutePathBuilder.get_path(
    f"05_{DATASET}_features",
    filepaths="../../../config/filepaths.yaml"
)

aae_terms_path = AbsolutePathBuilder.get_path(
    "00_aae_terms",
    filepaths="../../../config/filepaths.yaml"
)

In [3]:
terms = yaml.safe_load(open(aae_terms_path))["terms"]

In [4]:
def count_selected_terms(sentence, terms):
    count = 0
    for token in sentence.split(' '):
        if token in terms:
            count += 1
            
    return count

def calculate_aae_terms_count(input_path, terms):
    filenames = [file.split("/")[-1] for file in glob.glob(os.path.join(input_path, "*"))]
    for file in tqdm(filenames):
        df = pd.read_csv(os.path.join(input_path, file))
        df["text_wo_punctuation"] = df.text.str.replace(r"[^\w\d'\s]+", '', regex=True)

        df["AAE_TERMS_COUNT"] = df.text_wo_punctuation.apply(lambda s: count_selected_terms(s, terms))

        df.drop(columns=["text_wo_punctuation"], inplace=True)
        df.to_csv(os.path.join(input_path, file), index=False)    

In [5]:
calculate_aae_terms_count(data_path, terms)

100%|██████████████████████████████████████████████████████████████| 500/500 [00:01<00:00, 453.58it/s]
